In [1]:
# Check directory 
import os

print(os.getcwd())
print(os.listdir())

/Users/derphantomben/Documents/Coding/Portfolio
['NZ_earthquakes.csv', 'df_major_quake_cty.csv', 'df_major_quakes.csv', 'Untitled1.ipynb', '.DS_Store', 'MergeFile.ipynb', 'df_quake_cty.csv', 'EDA plots and visualization_2.ipynb', 'Untitled3.ipynb', 'Untitled.ipynb', 'Untitled4.ipynb', 'NZ_earthquakes_24.csv', 'Prep_Mod_MergeFile_2.ipynb', 'NewEDA-Copy1.ipynb', 'Untitled2.ipynb', 'frame_1.png', 'QuakePredict_2.ipynb', 'frame_3.png', 'NewEDA.ipynb', 'exported_plot_2.html', 'frame_2.png', 'Error displaying widget.docx', 'PB2002_boundaries.json', 'df_cities_cln.csv', 'exported_plot.html', 'QuakePredict.ipynb', 'NZ_Quakes.ipynb', 'df_quake_cty_24.csv', 'PortfolioKursOutline.pdf', '~$ror displaying widget.docx', '.ipynb_checkpoints', 'Draft_Portfolio', 'nz_cities.csv', 'EDA plots and visualization.ipynb']


In [2]:
############################################
### Import modules for EDA### 
############################################

# Basic operations and plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# for depth adjusted distance cpmnsidering depth
from scipy.spatial import cKDTree 
#to apply ST-DBSCAN
import hdbscan 
# Normalize temporal data (UnixTime)
from sklearn.preprocessing import StandardScaler

In [33]:
# Read earthquake data file
df_quakes = pd.read_csv('NZ_earthquakes_24.csv', low_memory=False) 

############################################
# First 5 rows of df
display(df_quakes)

,publicid,eventtype,origintime,modificationtime,longitude,latitude,magnitude,depth,magnitudetype,depthtype,...,evaluationstatus,evaluationmode,earthmodel,usedphasecount,usedstationcount,magnitudestationcount,minimumdistance,azimuthalgap,originerror,magnitudeuncertainty
0,2024p453984,earthquake,2024-06-16T23:22:53.506Z,2024-06-17T01:46:25.260Z,175.772568,-39.233845,1.031062,83.974358,MLv,NaN,...,confirmed,manual,iasp91,15,13,3,0.109392,49.579727,0.455124,0.690632
1,2024p453898,earthquake,2024-06-16T22:37:20.275Z,2024-06-17T01:28:59.405Z,176.509384,-40.145943,2.598690,36.212597,MLv,NaN,...,confirmed,manual,iasp91,31,22,26,0.097171,74.287148,0.337016,0.151588
2,2024p453894,earthquake,2024-06-16T22:34:42.955Z,2024-06-16T22:59:31.207Z,174.214157,-39.881680,1.622359,104.248863,MLv,NaN,...,confirmed,manual,iasp91,18,13,4,0.493551,111.963715,0.475502,0.359175
3,2024p453885,earthquake,2024-06-16T22:30:25.906Z,2024-06-17T01:42:38.517Z,176.168335,-38.031158,2.745529,226.374725,MLv,NaN,...,confirmed,manual,iasp91,20,14,5,0.824808,244.814529,0.699551,0.540183
4,2024p453851,earthquake,2024-06-16T22:12:17.103Z,2024-06-17T01:36:22.245Z,174.444122,-40.476860,2.475419,27.588871,MLv,NaN,...,confirmed,manual,iasp91,31,23,16,0.513217,88.549713,0.552167,0.113907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519459,1468935,earthquake,2000-01-01T07:37:23.078Z,2010-01-13T20:59:00.000Z,176.644070,-38.168910,1.838000,5.000000,ML,operator assigned,...,reviewed,manual,nz1dr,7,4,4,0.125000,135.000000,0.113830,0.283000
519460,1468933,earthquake,2000-01-01T07:16:58.734Z,2010-01-13T20:59:00.000Z,174.739330,-40.965730,2.039000,32.965800,ML,NaN,...,reviewed,manual,nz1dr,8,6,7,0.166000,178.000000,0.064240,0.174000
519461,1468917,earthquake,2000-01-01T04:29:06.088Z,2010-01-13T20:59:00.000Z,173.937740,-39.936440,3.565000,160.919400,ML,NaN,...,reviewed,manual,nz1dr,27,24,14,0.777000,179.000000,0.281390,0.323000
519462,1468906,earthquake,2000-01-01T02:41:34.420Z,2010-01-13T20:59:00.000Z,174.324390,-40.990290,2.132000,54.140200,ML,NaN,...,reviewed,manual,nz1dr,9,6,5,0.358000,132.000000,0.154520,0.165000


In [58]:
df_quakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519464 entries, 0 to 519463
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   Time               519464 non-null  datetime64[ns, UTC]
 1   UnixTime           519464 non-null  int64              
 2   Longitude          519464 non-null  float64            
 3   Latitude           519464 non-null  float64            
 4   Magnitude          519464 non-null  float64            
 5   Magnitude Type     519458 non-null  object             
 6   Depth              519464 non-null  float64            
 7   NZ_MMI             514263 non-null  category           
 8   Adjusted Distance  519464 non-null  float64            
 9   Nearest City       519464 non-null  object             
 10  Population         519464 non-null  int64              
 11  Cluster            519464 non-null  int64              
dtypes: category(1), datetime64[ns,

In [34]:
# Checking for number of rows and columns (size of df)

print('The number of rows is {}, while there are {} columns.'.format(*df_quakes.shape))

The number of rows is 519464, while there are 21 columns.


In [35]:
#Renaming Columns
############################################
df_quakes = df_quakes.rename(columns={'origintime': 'Time', 'longitude': 'Longitude', 'latitude': 'Latitude', 'depth': 'Depth', 
                                    'magnitude': 'Magnitude', 'magnitudetype': 'Magnitude Type'})

df_quakes.head().T

,0,1,2,3,4
publicid,2024p453984,2024p453898,2024p453894,2024p453885,2024p453851
eventtype,earthquake,earthquake,earthquake,earthquake,earthquake
Time,2024-06-16T23:22:53.506Z,2024-06-16T22:37:20.275Z,2024-06-16T22:34:42.955Z,2024-06-16T22:30:25.906Z,2024-06-16T22:12:17.103Z
modificationtime,2024-06-17T01:46:25.260Z,2024-06-17T01:28:59.405Z,2024-06-16T22:59:31.207Z,2024-06-17T01:42:38.517Z,2024-06-17T01:36:22.245Z
Longitude,175.772568,176.509384,174.214157,176.168335,174.444122
Latitude,-39.233845,-40.145943,-39.88168,-38.031158,-40.47686
Magnitude,1.031062,2.59869,1.622359,2.745529,2.475419
Depth,83.974358,36.212597,104.248863,226.374725,27.588871
Magnitude Type,MLv,MLv,MLv,MLv,MLv
depthtype,NaN,NaN,NaN,NaN,NaN


In [57]:
print('The df_quakes file has the following shape:', df_quakes.shape)
print('----------------------------------')
# Check dtypes for df_quake again
display(df_quakes.dtypes)

The df_quakes file has the following shape: (519464, 12)
----------------------------------


Time                 datetime64[ns, UTC]
UnixTime                           int64
Longitude                        float64
Latitude                         float64
Magnitude                        float64
Magnitude Type                    object
Depth                            float64
NZ_MMI                          category
Adjusted Distance                float64
Nearest City                      object
Population                         int64
Cluster                            int64
dtype: object

In [37]:
# 1.1 Remove the timezone offset from datetime strings
df_quakes['Time'] = df_quakes['Time'].str.rstrip('+00:00')

# 1.2 Convert to datetime
df_quakes['Time'] = pd.to_datetime(df_quakes['Time'], errors='coerce')

# 2. change Magnitude and Depth to float

df_quakes['Magnitude'] = df_quakes['Magnitude'].round(1)

df_quakes['Magnitude'] = df_quakes['Magnitude'].round(1)

# Check dtypes for df_quake_cty again
print('dtypes for df_quakes:')
display(df_quakes.dtypes)

dtypes for df_quakes:


publicid                              object
eventtype                             object
Time                     datetime64[ns, UTC]
modificationtime                      object
Longitude                            float64
Latitude                             float64
Magnitude                            float64
Depth                                float64
Magnitude Type                        object
depthtype                             object
evaluationmethod                      object
evaluationstatus                      object
evaluationmode                        object
earthmodel                            object
usedphasecount                         int64
usedstationcount                       int64
magnitudestationcount                  int64
minimumdistance                      float64
azimuthalgap                         float64
originerror                          float64
magnitudeuncertainty                 float64
dtype: object

In [38]:
# Checking for NaNs
############################################

# 1. Check the number of NaN values in each row
nan_counts = df_quakes.isnull().sum(axis=1)

# NaN values per row
print("Number of NaN values in each row:")
print(nan_counts)
print('----------------------------------')

# Total number of NaNs across all rows
total_nans_rows = nan_counts.sum()
print("Total number of NaN values across all rows: {}".format(total_nans_rows))
print('----------------------------------')

# Counts of NaN values per column
nan_counts_col = df_quakes.isnull().sum()
print("Number of NaN values in each column:")
print(nan_counts_col)
print('----------------------------------')

# Total number of NaNs across all columns
total_nans_columns = nan_counts_col.sum()
print('Total number of NaN values across all columns: {}'.format(total_nans_columns))
print('----------------------------------')

# Percentage of NaNs in the dataset
total_elements = df_major_quakes.size
percent_nans = (total_nans_columns / total_elements) * 100
print("Percentage of NaN values in the dataset: {:.2f}%".format(percent_nans))
print('----------------------------------\n')

Number of NaN values in each row:
0         1
1         1
2         1
3         1
4         1
         ..
519459    0
519460    1
519461    1
519462    1
519463    1
Length: 519464, dtype: int64
----------------------------------
Total number of NaN values across all rows: 603778
----------------------------------
Number of NaN values in each column:
publicid                      0
eventtype                106110
Time                          0
modificationtime              0
Longitude                     0
Latitude                      0
Magnitude                     0
Depth                         0
Magnitude Type                6
depthtype                383316
evaluationmethod           3983
evaluationstatus         106314
evaluationmode               66
earthmodel                 3983
usedphasecount                0
usedstationcount              0
magnitudestationcount         0
minimumdistance               0
azimuthalgap                  0
originerror                   0
magnitu

In [39]:
display(df_quakes.head().T)

,0,1,2,3,4
publicid,2024p453984,2024p453898,2024p453894,2024p453885,2024p453851
eventtype,earthquake,earthquake,earthquake,earthquake,earthquake
Time,2024-06-16 23:22:53.506000+00:00,2024-06-16 22:37:20.275000+00:00,2024-06-16 22:34:42.955000+00:00,2024-06-16 22:30:25.906000+00:00,2024-06-16 22:12:17.103000+00:00
modificationtime,2024-06-17T01:46:25.260Z,2024-06-17T01:28:59.405Z,2024-06-16T22:59:31.207Z,2024-06-17T01:42:38.517Z,2024-06-17T01:36:22.245Z
Longitude,175.772568,176.509384,174.214157,176.168335,174.444122
Latitude,-39.233845,-40.145943,-39.88168,-38.031158,-40.47686
Magnitude,1.0,2.6,1.6,2.7,2.5
Depth,83.974358,36.212597,104.248863,226.374725,27.588871
Magnitude Type,MLv,MLv,MLv,MLv,MLv
depthtype,NaN,NaN,NaN,NaN,NaN


In [40]:
# Check for duplicates in dataframe
############################################

duplicate_rows = df_quakes[df_quakes.duplicated()]

print('Duplicate rows check:')

# Display duplicate rows, if any
if not duplicate_rows.empty:
    print('Duplicate Rows:')
    display(duplicate_rows)
else:
    print('No duplicate rows found!')
    print('----------------------------------')

Duplicate rows check:
No duplicate rows found!
----------------------------------


In [41]:
# Summary statistics

df_quakes.describe().T

,count,mean,std,min,25%,50%,75%,max
Longitude,519464.0,164.329682,59.258298,-180.000000,173.500521,175.459955,176.622394,180.638540
Latitude,519464.0,-39.888455,4.265651,-69.912331,-41.624841,-39.875994,-38.550990,56.913818
Magnitude,519464.0,2.261626,1.386570,-9.000000,1.900000,2.300000,2.700000,8.300000
Depth,519464.0,47.772010,71.207328,-10.000000,10.156250,23.105469,48.067925,750.000000
usedphasecount,519464.0,18.304766,11.951566,0.000000,10.000000,16.000000,23.000000,215.000000
usedstationcount,519464.0,14.037514,10.538127,0.000000,7.000000,12.000000,17.000000,180.000000
magnitudestationcount,519464.0,8.945863,9.096376,0.000000,4.000000,7.000000,11.000000,171.000000
minimumdistance,519464.0,0.443874,1.532812,0.000000,0.097172,0.189633,0.405000,98.763779
azimuthalgap,519464.0,165.784471,79.323373,0.000000,101.586082,152.000000,222.000000,360.000000
originerror,519464.0,0.445222,7.147644,0.000000,0.188759,0.299939,0.490010,4470.413144


In [42]:
# Binning the 'Magnitude' column for categorization of magnitude ranges as per www.geonet.org.nz (=New Zealand Modified Mercalli Intensity Scale)
# Binning the data into discrete intervals and assign labels to each bin
bins = [0, 3, 4, 5, 6, 7, 8, 11]
labels = ['Unnoticeable', 'Weak', 'Light', 'Moderate', 'Strong', 'Severe', 'Extreme']

# Use pd.cut to categorize the 'Magnitude' column - categorical column named 'NZ_MMI' 
df_quakes['NZ_MMI'] = pd.cut(df_quakes['Magnitude'], bins=bins, labels=labels, right=False, include_lowest=True)


,Magnitude,NZ_MMI
0,1.0,Unnoticeable
1,2.6,Unnoticeable
2,1.6,Unnoticeable
3,2.7,Unnoticeable
4,2.5,Unnoticeable
...,...,...
519459,1.8,Unnoticeable
519460,2.0,Unnoticeable
519461,3.6,Weak
519462,2.1,Unnoticeable


In [43]:
# Convert UTC datetime to Unix time
df_quakes['UnixTime'] = df_quakes['Time'].astype('int64') // 10**9 
# '// 10**9' -> to convert nan-seconds to seconds
# int64 to get the number of nanoseconds since the Unix epoch

# Note: 
# Unix time format useful because it is a standardized format and Handling time zones, daylight saving time, 
# and other complexities of date and time management can be simplified by using Unix time

############################################

# Checking output
display(df_quakes[['Time', 'UnixTime']])

,Time,UnixTime
0,2024-06-16 23:22:53.506000+00:00,1718580173
1,2024-06-16 22:37:20.275000+00:00,1718577440
2,2024-06-16 22:34:42.955000+00:00,1718577282
3,2024-06-16 22:30:25.906000+00:00,1718577025
4,2024-06-16 22:12:17.103000+00:00,1718575937
...,...,...
519459,2000-01-01 07:37:23.078000+00:00,946712243
519460,2000-01-01 07:16:58.734000+00:00,946711018
519461,2000-01-01 04:29:06.088000+00:00,946700946
519462,2000-01-01 02:41:34.420000+00:00,946694494


In [16]:
# create df backup
df_quake_backup = df_quakes.copy()

#freeing Memory
import gc
gc.collect()

0

In [17]:
# Read cities data file (online resource: https://simplemaps.com/data/nz-cities)
df_cities = pd.read_csv('nz_cities.csv', low_memory=False) 

display(df_cities.head().T)

display(df_cities.dtypes)

,0,1,2,3,4
city,Auckland,Christchurch,Manukau City,Wellington,Northcote
lat,-36.8406,-43.531,-37.0,-41.2889,-36.8019
lng,174.74,172.6365,174.885,174.7772,174.7494
country,New Zealand,New Zealand,New Zealand,New Zealand,New Zealand
iso2,NZ,NZ,NZ,NZ,NZ
admin_name,Auckland,Canterbury,Auckland,Wellington,Auckland
capital,admin,admin,NaN,primary,NaN
population,1346091,383200,375600,216200,205605
population_proper,1346091,383200,375600,216200,205605


city                  object
lat                  float64
lng                  float64
country               object
iso2                  object
admin_name            object
capital               object
population             int64
population_proper      int64
dtype: object

In [18]:
df_cities.describe()

,lat,lng,population,population_proper
count,93.000000,93.000000,9.300000e+01,9.300000e+01
mean,-40.028875,170.324775,4.436963e+04,4.436963e+04
std,3.062823,36.428186,1.516684e+05,1.516684e+05
min,-46.413100,-176.561100,2.018000e+03,2.018000e+03
25%,-41.758100,172.666700,3.320000e+03,3.320000e+03
50%,-39.583300,174.850000,6.510000e+03,6.510000e+03
75%,-37.550000,175.611700,1.865100e+04,1.865100e+04
max,-35.112500,178.017800,1.346091e+06,1.346091e+06


In [19]:
############################################
# Checking df_cities for NaNs
############################################

# 1. Check the number of NaN values in each row
nan_cty_counts = df_cities.isnull().sum(axis=1)

# NaN values per row
print("Number of NaN values in each row:")
print(nan_counts)
print('----------------------------------')

# Total number of NaNs across all rows
total_cty_nans_rows = nan_cty_counts.sum()
print("Total number of NaN values across all rows: {}".format(total_cty_nans_rows))
print('----------------------------------')

# Counts of NaN values per column
nan_cty_counts_col = df_cities.isnull().sum()
print("Number of NaN values in each column:")
print(nan_cty_counts_col)
print('----------------------------------')

# Total number of NaNs across all columns
total_cty_nans_columns = nan_cty_counts_col.sum()
print('Total number of NaN values across all columns: {}'.format(total_cty_nans_columns))
print('----------------------------------')

# Percentage of NaNs in the dataset
total_cty_elements = df_cities.size
percent_cty_nans = (total_cty_nans_columns / total_cty_elements) * 100
print("Percentage of NaN values in the dataset: {:.2f}%".format(percent_cty_nans))
print('----------------------------------\n')

Number of NaN values in each row:
0        0
1        0
2        0
3        0
4        0
        ..
78452    0
78453    0
78454    0
78455    0
78456    0
Length: 78457, dtype: int64
----------------------------------
Total number of NaN values across all rows: 76
----------------------------------
Number of NaN values in each column:
city                  0
lat                   0
lng                   0
country               0
iso2                  0
admin_name            0
capital              76
population            0
population_proper     0
dtype: int64
----------------------------------
Total number of NaN values across all columns: 76
----------------------------------
Percentage of NaN values in the dataset: 9.08%
----------------------------------



In [20]:
duplicate_cty = sum(df_cities.duplicated())
print('The amount of duplicate values in df_cities:', duplicate_cty)

print('-'*50)

The amount of duplicate values in df_cities: 0
--------------------------------------------------


In [21]:
# Checking if the 'population' and 'population_proper' columns are equal as indicated in df_cities.head()
equal = df_cities['population'].equals(df_cities['population_proper'])
equal

True

In [22]:
df_cities_cln = df_cities.drop(columns=['iso2', 'admin_name', 'capital', 'population_proper'])
# removing 'iso2', as all cities under consideration are in NZ and a separate 'country' also exists
# removing 'admin_name' and 'capital' as administrative statues for analysis irrelevant
# removing 'population_proper' as it is a duplicate of 'population'

df_cities_cln.to_csv('df_cities_cln.csv')  # Save dataframe as df_cities_cln.csv in the current directory

df_cities_cln.head()

,city,lat,lng,country,population
0,Auckland,-36.8406,174.7400,New Zealand,1346091
1,Christchurch,-43.5310,172.6365,New Zealand,383200
2,Manukau City,-37.0000,174.8850,New Zealand,375600
3,Wellington,-41.2889,174.7772,New Zealand,216200
4,Northcote,-36.8019,174.7494,New Zealand,205605


In [23]:
df_cities_cln.shape

(93, 5)

In [24]:
df_cities_cln.dtypes

city           object
lat           float64
lng           float64
country        object
population      int64
dtype: object

In [44]:
############################################
# Create a cKDTree for cities to efficiently find the nearest city for each earthquake
############################################
cities_tree = cKDTree(df_cities_cln[['lat', 'lng']].values)

# Find the index of the nearest city for each earthquake event
_, nearest_city_indices = cities_tree.query(df_quakes[['Latitude', 'Longitude']].values)

# Retrieve the coordinates of the nearest cities using these indices
nearest_city_coords = df_cities_cln[['lat', 'lng']].iloc[nearest_city_indices].values

# Calculate the distances between earthquake events and the nearest city
distances_to_nearest_city = np.linalg.norm(nearest_city_coords - df_quakes[['Latitude', 'Longitude']].values, axis=1)

# Adjust distances based on earthquake depth
adjusted_distances = distances_to_nearest_city + np.abs(df_quakes['Depth'] - 10) * 0.1

# Add adjusted distances, nearest city, and population information to df_quake
df_quakes['Adjusted Distance'] = adjusted_distances
df_quakes['Nearest City'] = df_cities_cln.loc[nearest_city_indices, 'city'].values
df_quakes['Population'] = df_cities_cln.loc[nearest_city_indices, 'population'].values

# Check for missing population values
missing_population = df_quakes['Population'].isnull().sum()
if missing_population > 0:
    print(f"Warning: {missing_population} earthquakes do not have associated population information.")

# Display of the updated dataframe with nearest city, population, and adjusted distance
print("Updated DataFrame (df_quakes) with Nearest City, Population, and Adjusted Distance:")
display(df_quakes[['Time', 'Magnitude', 'Adjusted Distance', 'Nearest City', 'Population']])


Updated DataFrame (df_quakes) with Nearest City, Population, and Adjusted Distance:


,Time,Magnitude,Adjusted Distance,Nearest City,Population
0,2024-06-16 23:22:53.506000+00:00,1.0,7.644973,Turangi,3320
1,2024-06-16 22:37:20.275000+00:00,2.6,3.229114,Havelock North,13950
2,2024-06-16 22:34:42.955000+00:00,1.6,9.720313,Hawera,9792
3,2024-06-16 22:30:25.906000+00:00,2.7,21.772647,Rotorua,54204
4,2024-06-16 22:12:17.103000+00:00,2.5,2.437386,Paraparaumu Beach,9087
...,...,...,...,...,...
519459,2000-01-01 07:37:23.078000+00:00,1.8,0.588751,Kawerau,7080
519460,2000-01-01 07:16:58.734000+00:00,2.0,2.497397,Porirua,55218
519461,2000-01-01 04:29:06.088000+00:00,3.6,15.575392,Hawera,9792
519462,2000-01-01 02:41:34.420000+00:00,2.1,4.956427,Wellington,216200


In [53]:
# Create a copy of df_quake with the desired columns
df_quakes_cty = df_quakes[['Time', 'UnixTime', 'Longitude', 'Latitude', 'Magnitude', 'Magnitude Type', 'Depth', 'NZ_MMI', 'Adjusted Distance', 'Nearest City', 'Population']].copy()

# Display a sample of df_quake_cty to verify
print("Sample of df_quakes:")
display(df_quakes_cty.head())

Sample of df_quakes:


,Time,UnixTime,Longitude,Latitude,Magnitude,Magnitude Type,Depth,NZ_MMI,Adjusted Distance,Nearest City,Population
0,2024-06-16 23:22:53.506000+00:00,1718580173,175.772568,-39.233845,1.0,MLv,83.974358,Unnoticeable,7.644973,Turangi,3320
1,2024-06-16 22:37:20.275000+00:00,1718577440,176.509384,-40.145943,2.6,MLv,36.212597,Unnoticeable,3.229114,Havelock North,13950
2,2024-06-16 22:34:42.955000+00:00,1718577282,174.214157,-39.881680,1.6,MLv,104.248863,Unnoticeable,9.720313,Hawera,9792
3,2024-06-16 22:30:25.906000+00:00,1718577025,176.168335,-38.031158,2.7,MLv,226.374725,Unnoticeable,21.772647,Rotorua,54204
4,2024-06-16 22:12:17.103000+00:00,1718575937,174.444122,-40.476860,2.5,MLv,27.588871,Unnoticeable,2.437386,Paraparaumu Beach,9087


In [54]:
df_quakes_cty.dtypes

Time                 datetime64[ns, UTC]
UnixTime                           int64
Longitude                        float64
Latitude                         float64
Magnitude                        float64
Magnitude Type                    object
Depth                            float64
NZ_MMI                          category
Adjusted Distance                float64
Nearest City                      object
Population                         int64
dtype: object

In [55]:
############################################
# Dataset reduction to earthquake events --> need to cluster the data to identify earthquake events. 
  # Through this we can identify ‘anomalies’ (= major disruptive events) 
############################################

# Normalize spatial data
spatial_data = df_quakes_cty[['Latitude', 'Longitude']].values
spatial_scaler = StandardScaler()
spatial_data = spatial_scaler.fit_transform(spatial_data)

# Normalize temporal data using UnixTime
temporal_data = df_quakes_cty['UnixTime'].values.reshape(-1, 1)
temporal_scaler = StandardScaler()
temporal_data = temporal_scaler.fit_transform(temporal_data)

# Combine spatial and temporal data
combined_data = np.hstack((spatial_data, temporal_data))

# Apply ST-DBSCAN using hdbscan
clusterer = hdbscan.HDBSCAN(min_samples=10, min_cluster_size=100, metric='euclidean')
df_quakes_cty['Cluster'] = clusterer.fit_predict(combined_data)


In [56]:
############################################
# Filtering for clusters with magnitude => 5 events
############################################

# Defining a threshold for major events
magnitude_threshold = 5.0

# Filtering for major events based on the threshold within the clusters
major_events = df_quakes_cty[(df_major_quakes['Magnitude'] > magnitude_threshold) & (df_quakes['Cluster'] != -1)]

# Extract unique cluster labels from major events
major_events = major_events['Cluster'].unique()

# Filter df_quake_cty to include only the identified major event clusters
df_major_quakes = df_quakes_cty[df_quakes_cty['Cluster'].isin(major_events)]

# Display the DataFrame with major event clusters
display(df_major_quakes)

,Time,UnixTime,Longitude,Latitude,Magnitude,Magnitude Type,Depth,NZ_MMI,Adjusted Distance,Nearest City,Population,Cluster
6,2024-06-16 21:17:02.885000+00:00,1718572622,145.008865,19.403189,5.0,Mw(mB),100.000000,Moderate,70.402322,Kaitaia,5868,1
8,2024-06-16 19:43:44.300000+00:00,1718567024,-176.719513,-30.431572,3.7,Mw(mB),253.261673,Weak,37.846923,Waitangi,2018,12
19,2024-06-16 15:30:04.397000+00:00,1718551804,-179.818893,-37.200104,3.6,MLv,33.000000,Weak,9.796214,Waitangi,2018,7
24,2024-06-16 14:08:12.741000+00:00,1718546892,-179.120789,-21.524160,4.8,mB,638.672058,Light,85.440045,Waitangi,2018,9
25,2024-06-16 13:48:21.374000+00:00,1718545701,-177.396500,-20.054701,4.7,mB,512.176209,Light,74.128918,Waitangi,2018,9
...,...,...,...,...,...,...,...,...,...,...,...,...
289062,2012-01-02 09:25:38.854000+00:00,1325496338,-178.128876,-32.806717,4.7,M,181.058975,Light,28.360314,Waitangi,2018,12
289150,2012-01-01 20:08:34.069000+00:00,1325448514,-177.552673,-33.165325,4.5,M,33.000000,Light,13.131557,Waitangi,2018,12
289240,2012-01-01 11:58:59.636000+00:00,1325419139,-177.918060,-33.715019,4.7,M,20.655897,Light,11.391520,Waitangi,2018,12
289254,2012-01-01 07:49:53.984000+00:00,1325404193,-178.809112,-32.425018,4.2,M,298.734711,Light,40.617024,Waitangi,2018,12


In [43]:
df_quakes_cty.to_csv('df_quake_cty_24.csv', index=False)
df_major_quakes.to_csv('df_major_quakes.csv', index=False)